In [ ]:
# account to access namespaces and pods list and details
kubectl create serviceaccount cp-api-explorer
# ro permissions
kubectl create clusterrole ns-reader --verb=get,list,watch --resource=namespaces,pods
# bind role(permissions) to account
kubectl create clusterrolebinding ns-reader --clusterrole ns-reader --serviceaccount default:cp-api-explorer


[doc](https://nieldw.medium.com/curling-the-kubernetes-api-server-d7675cfc398c)

In [ ]:
# account has secret(s)
kubectl get serviceaccount cp-api-explorer -o json

In [ ]:
# list secrets
kubectl get secrets

In [ ]:
# we know which from SA, right
$serviceAccount = "cp-api-explorer"
# get right secret
$secretName = kubectl get serviceaccount cp-api-explorer -o json | ConvertFrom-Json `
    | % { $_.secrets } `
    | ? { $_.name.startsWith("$($serviceAccount)-token-") } `
    | select -First 1 | % { $_.name }
$secretName
# secret from secretName
$token = kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data.token | base64 -d }
# ca.crt
$caCrtFilename = "./k8sca.crt"
kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data."ca.crt" | base64 -d } | Out-File $caCrtFilename
#gc $caCrtFilename

In [ ]:
# k8s API endpoint
$server = kubectl -n default get endpoints kubernetes --no-headers | awk '{ print $2 }'
$apiServerUrl = "https://$($server)"
$apiServerUrl

In [ ]:
# try request
curl -s "$apiServerUrl/openapi/v2"  --header "Authorization: Bearer $token" --cacert $caCrtFilename

In [ ]:
/usr/bin/curl -v "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename

In [ ]:
$apiServerUrl="https://127.0.0.1:56533"

/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/kube-system/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename | ConvertTo-Json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 127.0.0.1...
* TCP_NODELAY set
* Connected to 127.0.0.1 (127.0.0.1) port 56533 (#0)
* ALPN, offering h2
* ALPN, offering http/1.1
* successfully set certificate verify locations:
*   CAfile: ./k8sca.crt
  CApath: none
* TLSv1.2 (OUT), TLS handshake, Client hello (1):
} [205 bytes data]
* TLSv1.2 (IN), TLS handshake, Server hello (2):
{ [64 bytes data]
* TLSv1.2 (IN), TLS handshake, Certificate (11):
{ [1002 bytes data]
* TLSv1.2 (IN), TLS handshake, Server key exchange (12):
{ [300 bytes data]
* TLSv1.2 (IN), TLS handshake, Request CERT (13):
{ [89 bytes data]
* TLSv1.2 (IN), TLS handshake, Server finished (14):
{ [4 bytes data]
* TLSv1.2 (OUT), TLS handshake, Certificate (11):
} [7 bytes data]
* TLSv1.2 (OUT), TLS handshake, Client key excha

In [ ]:
kubectl config
